In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import pyarrow.dataset as ds


### chargement des colonnes nécessaires pour optimiser la mémoire

In [3]:

codes_naf_numerique = [
    '62.01Z', '62.02A', '62.02B', '62.03Z', 
    '63.11Z', '63.12Z', '58.21Z', '58.29C', '58.29A', '58.29B',
]



### Filtrage : ESS + Codes NAF Numérique
L'objectif c'est de ne pas tout charger sinon l'ordi explose

In [4]:
dataset = ds.dataset('C:/Users/Utilisateur/OneDrive/Documents/centrale Lille + sc po/Projet SOGA/ess-numerique-database/data/raw/StockUniteLegale_utf8.parquet', format="parquet")

In [5]:
columns_to_extract = [
    'siren', 
    'denominationUniteLegale', 
    'nomUniteLegale', 
    'prenom1UniteLegale',
    'activitePrincipaleUniteLegale', 
    'categorieJuridiqueUniteLegale', 
    'trancheEffectifsUniteLegale',
    'dateCreationUniteLegale',      # Pour l'âge de la structure
    'identifiantAssociationUniteLegale', # Très utile pour le croisement ESS/Associations
    'nicSiegeUniteLegale'           # Permet de reconstruire le SIRET du siège social
]

table = dataset.to_table(
    columns=columns_to_extract,
)

### Recherche unique par code NAF


In [6]:
siren = '812479806'
t = dataset.to_table(columns=columns_to_extract, filter=(ds.field('siren') == siren))
df_siren = t.to_pandas()
if df_siren.empty:
    print(f"SIREN {siren} non trouvé dans le parquet.")
else:
    print(df_siren.to_string(index=False))

    siren denominationUniteLegale nomUniteLegale prenom1UniteLegale activitePrincipaleUniteLegale  categorieJuridiqueUniteLegale trancheEffectifsUniteLegale dateCreationUniteLegale identifiantAssociationUniteLegale  nicSiegeUniteLegale
812479806                DIGICOOP           None               None                        62.01Z                           5460                          03              2015-07-01                              None                   26


In [7]:
df_ess = table.to_pandas()

MemoryError: Unable to allocate 1.73 GiB for an array with shape (8, 29095403) and data type object

In [ ]:
print(f"Total d'acteurs ESS chargés : {len(df_ess)}")

Total d'acteurs ESS chargés : 1362312


In [ ]:
print("\nRépartition par activité (NAF) :")
print(df_ess['activitePrincipaleUniteLegale'].value_counts().head(10))


Répartition par activité (NAF) :
activitePrincipaleUniteLegale
94.99Z    623077
93.12Z    198455
90.01Z    123683
88.99B     51427
93.29Z     34410
93.19Z     25447
85.52Z     17918
85.59B     16622
90.02Z     16618
85.59A     14468
Name: count, dtype: int64


In [ ]:
print("\nTop 5 des catégories juridiques :")
print(df_ess['categorieJuridiqueUniteLegale'].value_counts().head(5))


Top 5 des catégories juridiques :
categorieJuridiqueUniteLegale
9220    1260608
9260      33959
6316      11690
5710      10856
9223      10706
Name: count, dtype: int64


In [ ]:
null_names = df_ess['denominationUniteLegale'].isnull().sum()
print(f"\nNombre d'entreprises sans dénomination sociale : {null_names}")


Nombre d'entreprises sans dénomination sociale : 0
